# Post-hoc evaluation

In [1]:
import os

import pandas as pd
import numpy as np
from datasets import load_dataset, load_metric

In [2]:
os.chdir('../..')

Predictions made by ALBERT during training

In [3]:
eval_predictions_df = pd.read_json('data/raw/example_eval_predictions.json', orient='index')
print(eval_predictions_df.shape)
eval_predictions_df.head()

(11873, 1)


,0
56ddde6b9a695914005b9628,France
56ddde6b9a695914005b9629,10th and 11th centuries
56ddde6b9a695914005b962a,"Denmark, Iceland and Norway"
56ddde6b9a695914005b962b,Rollo
56ddde6b9a695914005b962c,10th


In [4]:
eval_predictions_df.reset_index(inplace=True)
eval_predictions_df.rename(columns={'index': 'id', 0: "prediction_text"}, inplace=True)

In [5]:
eval_predictions_df

,id,prediction_text
0,56ddde6b9a695914005b9628,France
1,56ddde6b9a695914005b9629,10th and 11th centuries
2,56ddde6b9a695914005b962a,"Denmark, Iceland and Norway"
3,56ddde6b9a695914005b962b,Rollo
4,56ddde6b9a695914005b962c,10th
...,...,...
11868,5737aafd1c456719005744ff,
11869,5ad28ad0d7d075001a4299cc,
11870,5ad28ad0d7d075001a4299cd,
11871,5ad28ad0d7d075001a4299ce,


SQuAD 2 metric requires predictions and references (labels) provided in a specific way as described here: https://github.com/huggingface/datasets/blob/master/metrics/squad_v2/squad_v2.py#L52

`predictions`:

In [6]:
predictions = [
    {
        'prediction_text': prediction_text, 
        'id': _id, 
        'no_answer_probability': 0.0  # Same as https://github.com/huggingface/transformers/blob/master/examples/pytorch/question-answering/run_qa.py#L532
    } for 
    _id, prediction_text in zip(eval_predictions_df['id'], eval_predictions_df['prediction_text'])
]

`references`:

In [7]:
squad_v2_val = load_dataset('squad_v2', split='validation')

Reusing dataset squad_v2 (/Users/stevengeorge/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/ba48bc29b974701e9ba8d80ac94f3e3df924aba41b764dcf9851debea7c672e4)


In [8]:
squad_v2_val

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 11873
})

In [9]:
references = [
    {
        'id': _id, 
        'answers': answers
    } for 
    _id, answers in zip(squad_v2_val['id'], squad_v2_val['answers'])
]

In [10]:
metric = load_metric("squad_v2")

In [11]:
metric.compute(predictions=predictions, references=references)

{'exact': 82.39703529015414,
 'f1': 85.29448338798474,
 'total': 11873,
 'HasAns_exact': 79.60526315789474,
 'HasAns_f1': 85.40846849958548,
 'HasAns_total': 5928,
 'NoAns_exact': 85.18082422203533,
 'NoAns_f1': 85.18082422203533,
 'NoAns_total': 5945,
 'best_exact': 82.39703529015414,
 'best_exact_thresh': 0.0,
 'best_f1': 85.2944833879847,
 'best_f1_thresh': 0.0}

Matches values reported to wandb!

Can apply same process to subsets of DataFrame to get metrics by groups of examples.